In [7]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel
import torch
import subprocess
from pprint import pprint
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
import requests
import json
from pprint import pp

# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os
from openai import OpenAI
# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39312 entries, 0 to 39311
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   課程名稱      39312 non-null  object
 1   評分        25591 non-null  object
 2   評論數       25591 non-null  object
 3   Metadata  39312 non-null  object
 4   課程網址      39312 non-null  object
 5   課程        28384 non-null  object
 6   技能        38920 non-null  object
 7   課程資訊      33714 non-null  object
 8   師資        36153 non-null  object
 9   開課時間      33768 non-null  object
 10  建議學習時間    37339 non-null  object
 11  學習時長      29479 non-null  object
dtypes: object(12)
memory usage: 3.6+ MB


In [4]:
df.head()

,課程名稱,評分,評論數,Metadata,課程網址,課程,技能,課程資訊,師資,開課時間,建議學習時間,學習時長
0,Introduction to Artificial Intelligence (AI),4.7 out of 5 stars,20K reviews,Beginner · Course · 1 - 4 Weeks,https://www.coursera.org/learn/introduction-to-ai,4 modules,"Data Governance, Business Ethics, ChatGPT, Com...",Introduction and Applications of AI | AI Conce...,Rav Ahuja,Starts Jul 17,Flexible schedule,Approx. 12 hours.Learn at your own pace
1,AI For Everyone,4.8 out of 5 stars,48K reviews,Beginner · Course · 1 - 4 Weeks,https://www.coursera.org/learn/ai-for-everyone,4 modules,"Data Ethics, Strategic Thinking, Team Building...",What is AI? | Building AI Projects | Building ...,Andrew Ng,Starts Jul 17,Flexible schedule,Approx. 6 hours.Learn at your own pace
2,Introduction to AI,4.9 out of 5 stars,1.7K reviews,Beginner · Course · 1 - 4 Weeks,https://www.coursera.org/learn/google-introduc...,1 module,"Innovation, Data Quality, Machine Learning Sof...",Introduction to AI,NaN,Starts Jul 17,Flexible schedule,Approx. 1 hourLearn at your own pace
3,Machine Learning,4.9 out of 5 stars,35K reviews,Beginner · Specialization · 1 - 3 Months,https://www.coursera.org/specializations/machi...,NaN,"Artificial Intelligence, Predictive Modeling, ...",Supervised Machine Learning: Regression and Cl...,"Andrew Ng, Geoff Ladwig, Aarti Bagul, Eddy Shy...",Starts Jul 17,2 months,at 10 hours. a week
4,Generative AI: Prompt Engineering Basics,4.8 out of 5 stars,5.1K reviews,Beginner · Course · 1 - 4 Weeks,https://www.coursera.org/learn/generative-ai-p...,3 modules,"Large Language Modeling, IBM Cloud, Prompt Eng...",Prompt Engineering for Generative AI | Prompt ...,"Antonio Cangiano, Rav Ahuja",Starts Jul 17,NaN,NaN


In [29]:
df = pd.read_csv(r"C:\Users\user\Desktop\期末專題\coursera_merged.csv")

df_sample = df.sample(n=100, replace=False, random_state=42)

df_sample["課程相關總描述"] = (
    df_sample["課程名稱"].astype(str) + "\n" +
    df_sample["技能"].astype(str) + "\n" +
    df_sample["課程資訊"].astype(str)
)
df_sample.info()


<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 38483 to 30264
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   課程名稱      100 non-null    object
 1   評分        68 non-null     object
 2   評論數       68 non-null     object
 3   Metadata  100 non-null    object
 4   課程網址      100 non-null    object
 5   課程        72 non-null     object
 6   技能        98 non-null     object
 7   課程資訊      89 non-null     object
 8   師資        92 non-null     object
 9   開課時間      89 non-null     object
 10  建議學習時間    94 non-null     object
 11  學習時長      76 non-null     object
 12  課程相關總描述   100 non-null    object
dtypes: object(13)
memory usage: 10.9+ KB


In [32]:
df_sample["課程相關總描述"]

38483    大數據分析：商業應用與策略管理 (Big Data Analytics: Business ...
23266    Generative AI for Sales Professionals\nCustome...
17365    مهارات ذكاء النجاح | Success Intelligence Skil...
4737     Computer Science: Programming with a Purpose\n...
13626    A prática da gestão de clubes e federações esp...
                               ...                        
36331    Flow Forward: Mastering User Journeys & Wirefr...
22001    Excel/VBA for Creative Problem Solving, Part 3...
22233    3D Models for Virtual Reality\nGame Design, Vi...
12335    AstroTech: The Science and Technology behind A...
30264    Applied Machine Learning in Python\nMachine Le...
Name: 課程相關總描述, Length: 100, dtype: object

In [103]:
def extract_skills_gpt(text, model="gpt-4.1-nano"):
    prompt = f"""
你現在是一位資深的軟體工程師，
熟悉市面上全部跟軟體相關的技能。
也是一個有多年教軟體課程經驗的老師，
用你的專業去判斷以下的課程內容，
並從中列出上課會學到的技能或工具，並用分號分隔。
並在工具或技能名稱後面附上這堂課會學得這個工具的哪些部分。

請遵守以下規則：
1. 不要包含任何軟實力（如：溝通能力、團隊合作）
2. 使用繁體中文及英文回覆
3. 回復範例如下：
資料科學（Data Science）:學習資料收集、探索性分析 (EDA) 與特徵工程在 AI 專案中的角色;
人工神經網路（Artificial Neural Networks）:掌握神經元結構、前向傳播與反向傳播的基本原理;
...

【注意】請產出內容時特別檢查技能或工具之間是否都有用";"做區隔。
以下是職缺內容：
{text}
"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=512,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"錯誤：{e}"
# ✅ 套用 GPT 抽取技能（你可以先測試前幾筆）
df_sample["課程技能總覽"] = df_sample["課程相關總描述"].astype(str).apply(extract_skills_gpt)
df_sample.to_csv("output_skills.csv", index=False, encoding="utf-8-sig")

In [106]:
df_sample["課程網址"].iloc[32]


'https://www.coursera.org/learn/react-basics'

In [107]:
pp(df_sample["課程技能總覽"].iloc[32])

('React 基礎（React Basics）:學習 React.js 的基本概念與架構;  \n'
 '超文本標記語言（Hypertext Markup Language, HTML）:掌握網頁結構與元素標記;  \n'
 'CSS（Cascading Style Sheets）:學習樣式設計與佈局調整;  \n'
 'JavaScript（Javascript）:掌握基本語法、事件處理與資料操作;  \n'
 '事件驅動程式設計（Event-Driven Programming）:理解事件監聽與回呼函數;  \n'
 '前端網頁開發（Front-End Web Development）:整合 HTML、CSS、JavaScript 進行前端設計;  \n'
 'Web 應用程式（Web Applications）:了解 Web 應用的架構與流程;  \n'
 '開發環境（Development Environment）:設定與使用開發工具與流程;  \n'
 '移動端開發（Mobile Development）:理解 React 在行動裝置上的應用;  \n'
 '應用程式開發（Application Development）:掌握 React 建立完整應用的流程;  \n'
 'UI 元件（UI Components）:設計與使用 React 元件模組;  \n'
 'React 元件（React Components）:建立與管理可重用的 UI 元件;  \n'
 '資料與狀態（Data and State）:管理元件資料流與狀態變化;  \n'
 '導航（Navigation）:實作多頁面或多元內容的導覽功能;  \n'
 '更新與資產（Updating and Assets in React.js）:處理資料更新、資源載入與管理;  \n'
 '第一個 React 應用（Your first React app）:從零建立並運行第一個 React 應用程式;')


In [115]:
df_split = df_sample["課程技能總覽"].str.split(";")

# 1. 依分號切開並展開為多欄
df_skills = (
    df_sample["課程技能總覽"]
    .fillna("")                              # 避免 None/NaN 出錯
    .str.split(";", expand=True)             # 直接水平展開
    .rename(columns=lambda i: f"技能{i+1}")  # 重新命名欄位
)

# 2. 重設索引，保證對齊
df_sample  = df_sample.reset_index(drop=True)
df_skills  = df_skills.reset_index(drop=True)

# 3. 水平合併
df_result = df_sample.join(df_skills)        # 或 pd.concat([df_sample, df_skills], axis=1)

# 4. (選) 若不再需要原文字欄位，可刪除
# df_result = df_result.drop(columns=["課程技能總覽"])

# 檢視
df_result.head()

,課程名稱,評分,評論數,Metadata,課程網址,課程,技能,課程資訊,師資,開課時間,...,技能34,技能35,技能36,技能37,技能38,技能39,技能40,技能41,技能42,技能43
0,大數據分析：商業應用與策略管理 (Big Data Analytics: Business ...,4.8 out of 5 stars,405 reviews,Beginner · Course · 1 - 3 Months,https://www.coursera.org/learn/bigdataanalysis,7 modules,"Retail Store Operations, Data-Driven Decision-...",èª²ç¨ç°¡ä» | æ¸æåæå¨éèåè²¡åä...,National Taiwan University,Starts Jul 19,...,None,None,None,None,None,None,None,None,None,None
1,Generative AI for Sales Professionals,4.4 out of 5 stars,12 reviews,Beginner · Specialization · 1 - 3 Months,https://www.coursera.org/specializations/micro...,NaN,"Customer Data Management, Sales Strategy, Cust...","Copilot for Sales | Customer Data, Analytics, ...",NaN,Starts Jul 19,...,None,None,None,None,None,None,None,None,None,None
2,مهارات ذكاء النجاح | Success Intelligence Skills,4.9 out of 5 stars,17 reviews,Beginner · Course · 1 - 4 Weeks,https://www.coursera.org/learn/success-intelli...,1 module,"Organizational Effectiveness, Growth Mindednes...",Ù ÙØ§Ø±Ø§Øª Ø°ÙØ§Ø¡ Ø§ÙÙØ¬Ø§Ø­,Alfaisal University | KLD,Starts Jul 18,...,None,None,None,None,None,None,None,None,None,None
3,Computer Science: Programming with a Purpose,4.7 out of 5 stars,1.3K reviews,Beginner · Course · 1 - 3 Months,https://www.coursera.org/learn/cs-programming-...,10 modules,"Object Oriented Programming (OOP), Computer Pr...",BASIC PROGRAMMING CONCEPTS | CONDITIONALS AND ...,"Robert Sedgewick, Kevin Wayne, Princeton Unive...",Starts Jul 19,...,None,None,None,None,None,None,None,None,None,None
4,A prática da gestão de clubes e federações esp...,4.8 out of 5 stars,274 reviews,Beginner · Course · 1 - 3 Months,https://www.coursera.org/learn/fia-gestao-club...,6 modules,"Event Management, Marketing Management, Advert...",IntroduÃ§Ã£o ao Sistema Esportivo Brasileiro |...,"Michel Fauze Mattar, FundaÃ§Ã£o Instituto de A...",Starts Jul 18,...,None,None,None,None,None,None,None,None,None,None


In [119]:
# 篩選欄位名稱中包含「技能」的欄位
skill_columns = [col for col in df_result.columns if "技能" in col]

# 顯示這些欄位
df_result[skill_columns]
# df_result.info()


,技能,課程技能總覽,技能1,技能2,技能3,技能4,技能5,技能6,技能7,技能8,...,技能34,技能35,技能36,技能37,技能38,技能39,技能40,技能41,技能42,技能43
0,"Retail Store Operations, Data-Driven Decision-...",大數據分析（Big Data Analytics）:學習資料收集與處理 (Data Coll...,大數據分析（Big Data Analytics）:學習資料收集與處理 (Data Coll...,資料探索與視覺化 (Exploratory Data Analysis and Visual...,資料模型建立與評估 (Model Building and Evaluation),商業應用策略制定 (Business Application Strategies),策略管理與決策支持 (Strategic Decision-Making Support),資料庫與資料倉儲技術 (Database and Data Warehouse Techno...,資料清洗與轉換 (Data Cleaning and Transformation),資料分析工具與平台 (Data Analysis Tools and Platforms)如...,...,None,None,None,None,None,None,None,None,None,None
1,"Customer Data Management, Sales Strategy, Cust...",客戶資料管理（Customer Data Management）: 學習資料整理、資料庫設計...,客戶資料管理（Customer Data Management）: 學習資料整理、資料庫設計...,\n銷售策略（Sales Strategy）: 掌握策略規劃、目標設定與策略執行流程,\n客戶洞察（Customer Insights）: 進行資料分析、行為分析與客戶細分,\n銷售啟用（Sales Enablement）: 了解銷售工具整合與內容管理,\n銷售簡報（Sales Presentation）: 學習簡報設計與資料視覺化技巧,\n生成式AI代理（Generative AI Agents）: 掌握生成式AI模型運作...,\n微軟Outlook（Microsoft Outlook）: 熟悉郵件管理、行事曆整合...,\n銷售（Sales）: 了解銷售流程、銷售管理系統與銷售數據分析,...,None,None,None,None,None,None,None,None,None,None
2,"Organizational Effectiveness, Growth Mindednes...",成功智慧（Success Intelligence）:學習組織效能提升技巧; \n組織效能...,成功智慧（Success Intelligence）:學習組織效能提升技巧,\n組織效能（Organizational Effectiveness）:理解組織運作與...,\n成長心態（Growth Mindedness）:培養持續學習與適應變化的思維模式,\n領導力（Leadership）:掌握領導技巧、激勵團隊與決策制定,\n商業領導（Business Leadership）:理解企業策略制定與執行,\n協作（Collaboration）:學習團隊合作與跨部門協調技巧,\n主動性與領導（Initiative and Leadership）:培養主動解決問題...,\n目標設定（Goal Setting）:掌握SMART目標制定與追蹤方法,...,None,None,None,None,None,None,None,None,None,None
3,"Object Oriented Programming (OOP), Computer Pr...","物件導向程式設計（Object Oriented Programming, OOP）:理解類...","物件導向程式設計（Object Oriented Programming, OOP）:理解類...",\n電腦程式設計（Computer Programming）:掌握基本的程式撰寫流程與邏輯結構,\n演算法（Algorithms）:學習演算法設計、分析與最佳化技巧,\n動畫（Animations）:應用圖形與動畫技術於程式中,\nJava程式設計（Java Programming）:撰寫Java語言的程式碼、理解...,\nJava語言（Java）:掌握Java語言的基本語法、物件導向特性與應用範例,\n資料結構（Data Structures）:學習陣列、鏈結串列、堆疊、佇列、樹、圖等...,\n效能調校（Performance Tuning）:分析與優化程式效能，包括記憶體管理...,...,None,None,None,None,None,None,None,None,None,None
4,"Event Management, Marketing Management, Advert...",資料管理與分析（Data Management & Analysis）:學習資料收集、財務管...,資料管理與分析（Data Management & Analysis）:學習資料收集、財務管...,\n行銷與廣告（Marketing & Advertising）:掌握行銷策略、媒體關係...,\n策略規劃（Strategic Planning）:理解組織結構、商業策略、經濟分析在...,\n財務管理（Financial Management）:學習財務規劃、預算控制、資金運...,\n媒體關係（Media Relations）:掌握媒體溝通策略、新聞稿撰寫、媒體合作技巧,\n體育系統管理（Sports System Management）:理解巴西體育系統架...,\n贊助策略（Sponsorship Management）:學習贊助協商、贊助方案設計...,\n組織管理（Organizational Structure）:掌握體育組織架構設計、...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Usability, User Interface (UI), User Experienc...",用戶體驗設計（User Experience Design）: 理解用戶需求、設計符合用戶行...,用戶體驗設計（User Experience Design）: 理解用戶需求、設計符合用戶行...,\n用戶流程（User Flows）: 規劃用戶在產品中的操作路徑與交互流程,\n線框圖（Wireframing）: 製作低保真界面原型以驗證設計概念,\nFigma（設計軟體）: 使用Figma進行界面設計、線框圖製作與協作,\n可用性（Usability）: 評估產品的易用性與用戶互動效率,\n用戶界面（UI）設計（User Interface Design）: 設計直觀且吸引...,\n用戶研究（User Research）: 進行用戶需求分析與行為觀察,\n用戶中心設計（User Centered Design）: 以用戶需求為核心進行設計流程,...,None,None,None,None,None,None,None,None,None,None
96,"Visual Basic (Programming Language), Debugging...",Excel（Microsoft Excel）:資料管理與分析技巧;資料篩選、資料整理、數據視...,Excel（Microsoft Excel）:資料管理與分析技巧,資料篩選、資料整理、數據視覺化、資料驗證 \nVBA（Visual Basic for A...,自動化任務、建立自訂函數、資料處理流程 \nVisual Basic（Programmin...,建立用戶界面、撰寫邏輯控制、除錯技巧 \nDebugging（除錯技巧）:除錯流程、錯誤追...,找出並修正程式錯誤、提升程式穩定性 \nSimulations（模擬）:蒙地卡羅模擬、隨機...,建立模擬模型、數據生成與分析、結果驗證 \nUser Interface（UI）設計:界面...,設計直觀界面、提升用戶體驗 \nAutomation（自動化）:流程自動化、資料更新、報表產生,提升工作效率、減少手動操作 \nPeer Review（同行評審）:程式碼檢查、最佳實踐分...,...,None,None,None,None,None,None,None,None,None,None
97,"Game Design, Virtual Reality, Computer Graphic...",3D模型設計（3D Modeling）:學習建模技巧、材質貼圖與細節雕刻; \n虛擬實境開...,3D模型設計（3D Modeling）:學習建模技巧、材質貼圖與細節雕刻,\n虛擬實境開發（Virtual Reality Development）:掌握VR專用...,\n遊戲設計（Game Design）:理解遊戲玩法、關

In [121]:
df_result= df_result.drop("課程技能總覽", axis=1)
df_result.to_csv("coursera_skills_1140721.csv", index=False, encoding="utf-8-sig")